In [9]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [10]:
class Universidades:

    # Creamos la clase sin método constructor, ya que no necesitamos realizar ninguna inicialización especial.

    def __init__(self):
        pass

    # 1. Extracción

    def api(self, paises):

        """Esta función recibe un único parámetro, que será un string o una lista de strings con el nombre del país o países (en inglés) de los que se quiere
        obtener información. Se realizará la extracción de los datos de los países indicados, y devolverá un DataFrame
        con la información obtenida. En caso de que hubiera algún error, devolverá el número del error y el motivo."""

        if isinstance(paises, str): # Comprueba si el parámetro introducido es un string. 

            url = f"http://universities.hipolabs.com/search?country={paises}" 

            response = requests.get(url)

            code = response.status_code # Devuelve el código de la extracción de datos

            reason = response.reason # Devuelve el motivo.

            if code == 200: # Comprueba que el código de la extracción sea 200, es decir, que haya ocurrido correctamente.

                print(f"Información de {paises} obtenida correctamente, se va a convertir en DataFrame.")

                df =  pd.json_normalize(response.json())

                return df # Devuelve el DataFrame con la información del país indicado. 
            
            else: 
                return f"Error: {code}, {reason}." # Si no, devuelve el código de error y el motivo.
        
        elif isinstance(paises, list):  # Comprueba si el parámetro introducido es una lista.

            df_universidades = pd.DataFrame() #Se crea un DataFrame vacío para ir uniendo los resultados por cada país de la lista introducida.

            for pais in paises: 

                url = f"http://universities.hipolabs.com/search?country={pais}"

                response = requests.get(url) 

                code = response.status_code # Devuelve el código de la extracción de datos

                reason = response.reason # Devuelve el motivo.

                if code == 200: # Comprueba que el código de la extracción sea 200, es decir, que haya ocurrido correctamente.

                    print(f"Información de {pais} obtenida correctamente, se va a convertir en DataFrame.")

                    df =  pd.json_normalize(response.json())

                    df_universidades = pd.concat([df_universidades, df], axis = 0) # Se unen los resultados en el DataFrame creado

                else: 
                    return f"Error: {code}, {reason}."
                
            
            return df_universidades # Devuelve el DataFrame con la información de los paises indicados si se ha podido crear. Fuera del bucle para
                                    # que se pueda completar el bucle. 
        
        else:
            print("Por favor, introduzca un único país o una lista de paises (en inglés).")

    
    # 2. Limpieza

    def limpieza(self, dataframe):

        """Esta función recibe como parámetro el nombre del dataframe a limpiar.
            Devuelve el dataframe con el nombre de las columnas cambiadas ("-" ahora es "_"), y elimina una columna redundante. Además,
            realiza el explode a la columna "web_pages, elimina los duplicados de la columna  "name", imputa los nulos de
            la columna "state_province" por la categoría "Unknown". Asimismo, cambia los estados por el nombre completo. 
            Devuelve el dataframe con los datos aplicados. """

        nuevas_columnas = {col : col.replace("-", "_") for col in dataframe.columns} # Se crea un diccionario con los nombres antiguos como key, y 
                                                                                    #los nuevos nombres como value.

        dataframe.rename(columns = nuevas_columnas, inplace = True) # Se realiza el cambio del nombre de las columnas.
            
        dataframe.drop("domains", axis = 1, inplace = True) # Se elimina la columna de domains.

        dataframe = dataframe.explode("web_pages") # Se realiza el explode para la columna de "web_pages"

        dataframe.drop_duplicates(subset = "name", inplace = True) # Se eliminan los duplicados en la columna "name"

        dataframe["state_province"].fillna("Unknown", inplace = True)

        dataframe["state_province"] = dataframe["state_province"].replace("NV", "Nevada").replace("TX", "Texas").replace("IN", "Indianapolis").replace("CA", "California").replace("VA", "Virginia").replace("NY", "New York")
            
        dataframe["state_province"] = dataframe["state_province"].replace("New York, NY", "New York").replace("MI", "Michigan").replace("GA", "Georgia").replace("ND", "North Dakota").replace("Ciudad Autónoma de Buenos Aires", "Buenos Aires")
        
        # Se realizan varias modificaciones a la columna "state_province", imputando los nulos por una nueva categoría y reemplazando los nombres de estados
        # por otros nombres más claros. El replace se podría haber hecho una única línea de código, pero quedaba muy larga, así que he 
        # optado por partirla en dos. 
        
        dataframe["name"] = dataframe["name"].str.replace('"', '') # Hay algunas universidades que tienen dobles comillas en sus nombres, lo que impide su inserción en la base de datos. 
                                                                    #Lo solucionamos aquí para evitar problemas en la futura inserción
    

        return dataframe # Se devuelve el dataframe con los cambios aplicados 
        
    def sacar_coordenadas(self, dataframe):
        
        """Esta función recibe dos parámetros, que es el nombre del dataframe y el nombre de la columna sobre la que se quieren sacar las coordenadas.
        Se utiliza la librería de Geopy para conseguir la latitud y la longitud de los lugares indicados. 
        Devuelve el DataFrame con dos columnas nuevas, "lat" y "long", que contienen las coordenadas en función del lugar de la columna indicada."""

        lista_estados = list(dataframe["state_province"].unique()) # Primero, se crea una lista con los valores únicos de esa columna.

        df_localizacion = pd.DataFrame(columns = ["state_province", "lat", "long"]) # Se crea un datagrame vacío para poder introducir los datos. 

        for estado in lista_estados: 

            if estado != "Unknown":  
                geo = Nominatim(user_agent = "nombre")
                localizacion = geo.geocode(f"{estado}")
                state = f"{estado}"
                latitud = localizacion[1][0]
                longitud = localizacion[1][1]

                df_localizacion.loc[len(df_localizacion.index)] = [state, latitud, longitud] #Se añaden los datos sacados de Geopy en la última fila del DataFrame creado.
            
            else: # Si el estado/lugar entra en la categoría creada previamente de "Unknown", entonces las coordenadas se convertirán a un nulo de NumPY.
                
                state = f"{estado}"
                latitud = np.nan
                longitud = np.nan
                
                df_localizacion.loc[len(df_localizacion.index)] = [state, latitud, longitud]
            
        dataframe = pd.merge(dataframe, df_localizacion, on = "state_province", how = "left")

        return dataframe
    
    def guardar_df(self, dataframe, ruta_nombre):

        """Esta función recibe dos parámetros, que son el nombre del dataframe a guardar y la ruta donde se quiere guardar. 
        Guarda el archivo tanto en .pkl como en .csv"""

        dataframe.to_csv(f"{ruta_nombre}.csv")
        dataframe.to_pickle(f"{ruta_nombre}.pkl")

In [11]:
df = Universidades()

In [12]:
lista_paises = ["Argentina", "Canada", "United States"]

In [13]:
df_universidades = df.api(lista_paises)

Información de Argentina obtenida correctamente, se va a convertir en DataFrame.
Información de Canada obtenida correctamente, se va a convertir en DataFrame.
Información de United States obtenida correctamente, se va a convertir en DataFrame.


In [14]:
df_universidades

,web_pages,alpha_two_code,state-province,name,domains,country
0,[http://www.atlantida.edu.ar/],AR,Buenos Aires,Universidad Atlantida Argentina,[atlantida.edu.ar],Argentina
1,[http://www.austral.edu.ar/],AR,Buenos Aires,Universidad Austral Buenos Aires,[austral.edu.ar],Argentina
2,[http://www.caece.edu.ar/],AR,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",[caece.edu.ar],Argentina
3,[http://www.cema.edu.ar/],AR,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,[cema.edu.ar],Argentina
4,[http://www.iese.edu.ar/],AR,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,[iese.edu.ar],Argentina
...,...,...,...,...,...,...
4531,[https://csu.edu],US,None,Chicago State University,[csu.edu],United States
4532,[https://csupueblo.edu],US,None,Colorado State University-Pueblo,[csupueblo.edu],United States
4533,[https://bau.edu/],US,None,Bay Atlantic University,[bau.edu],United States
4534,[https://www.siena.edu/],US,None,Siena College,[siena.edu],United States


In [15]:
df_universidades = df.limpieza(df_universidades)

In [16]:
df_universidades = df.sacar_coordenadas(df_universidades)

In [17]:
df.guardar_df(df_universidades, "../data/universidades")